In [1]:
#clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15232, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15232 (delta 22), reused 32 (delta 19), pack-reused 15185
Receiving objects: 100% (15232/15232), 13.66 MiB | 21.13 MiB/s, done.
Resolving deltas: 100% (10332/10332), done.


In [2]:
%cd darknet

/content/darknet


In [3]:
#change makefile to have OPENCV, GPU, CUDNN enabled
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
#compile Darknet
!make

In [ ]:
#get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

In [8]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import IPython
import time
import math

In [9]:
min_confidence = 0.5

weight_file = 'yolov3.weights'
cfg_file = 'cfg/yolov3.cfg'
name_file = 'data/coco.names'

#https://bdd-data.berkeley.edu/ 에서 dataset download
#http://dl.yf.io/bdd100k/video_parts/bdd100k_videos_test_00.zip의 첫번째 video file
file_name = 'cabc30fc-e7726578.mp4'

In [10]:
from google.colab import files
files.upload()

Saving cabc30fc-e7726578.mp4 to cabc30fc-e7726578.mp4


In [11]:
#load yolo
net = cv2.dnn.readNet(weight_file, cfg_file)

In [12]:
classes = []
with open(name_file, 'r') as f:
  classes = [line.strip() for line in f.readlines()]
print(classes)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [14]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [15]:
def writeFrame(img):
  #use global variable, writer
  global writer
  height, width  = img.shape[:2]
  if writer is None and output_name is not None:
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(output_name, fourcc, 24, (width, height), True)
  if writer is not None:
    writer.write(img)

In [ ]:
frame_count = 0

#initialize the video writer
writer = None
output_name = 'output_video.avi'

file_name = 'cabc30fc-e7726578.mp4'
face_cascade_name = 'haarcascade_frontalface_alt.xml'

detected = False
frame_mode = 'Tracking'
elapsed_time = 0

tracker = cv2.TrackerKCF_create()
trackers = cv2.MultiTracker_create()

vs = cv2.VideoCapture(file_name)

while True:
  IPython.display.clear_output(wait=True)
  ret, frame = vs.read()
  if frame is None:
    print('no more frame')
    break
  
  start_time = time.time()
  frame_count += 1

  if detected:
    frame_mode = 'Tracking'
    (success, boxes) = trackers.update(frame)
    for box in boxes:
      (x, y, w, h) = [int(v) for v in box]
      cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255,0), 2)
  else:
    frame_mode = 'Detection'
    
    height, width, channels = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0,0,0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
      for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        #filter only car
        if (confidence > min_confidence) and class_id == 2:
          #object detected
          center_x = int(detection[0]*width)
          center_y = int(detection[1]*height)
          w = int(detection[2]*width)
          h = int(detection[3]*height)

          #rectangle coordinates
          x = int(center_x - w / 2)
          y = int(center_y - h / 2)

          boxes.append([x, y, w, h])
          confidences.append(float(confidence))
          class_ids.append(class_id)

    #Region of interest

    roi_left = int(0.3 * width)
    roi_right = int(0.6 * width)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_COMPLEX

    for i in range(len(boxes)):
      if i in indexes:
        x, y, w, h = boxes[i]
        
        #eliminate small object( < 50 )
        if ( w > 50 ) and ( x > roi_left and x < roi_right):
          label = str(classes[class_ids[i]])
          print(class_ids[i], label, w)
          color = (0, 255, 0)
          selected = boxes[i]
          cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)
    
    trackers.add(tracker, frame, tuple(selected[0]))
    detected = True

    cv2_imshow(frame)
    frame_time = time.time() - start_time
    elapsed_time += frame_time
    print("[{}] Frame {} time {}".format(frame_mode, frame_count, frame_time))

    writeFrame(frame)
  
print("elapsed time {}".format(elapsed_time))
vs.release()
